In [2]:
import numpy as np 
import pandas as pd
import os
import shutil
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

# Pega todo caminho
base_dir = os.getcwd()
# Pega o caminho e atribui as variaveis
CincoCentDir = os.path.join(base_dir,'dataset/5/') 
DezCentDir = os.path.join(base_dir,'dataset/10/') 
VinteCincoCentDir = os.path.join(base_dir,'dataset/25/') 
CinquentaCentDir = os.path.join(base_dir,'dataset/50/') 
UmRealCentDir = os.path.join(base_dir,'dataset/100/')  

# verificar se esta carreando todas as imagens
cincoCent_images = os.listdir(CincoCentDir)
dezCent_images = os.listdir(DezCentDir)
vinteCincoCent_images = os.listdir(VinteCincoCentDir)
cinquentaCent_images = os.listdir(CinquentaCentDir)
umRealCent_images = os.listdir(UmRealCentDir)

print('quantidade de imagem em cada cada pasta respectivamente:',len(cincoCent_images), len(dezCent_images), len(vinteCincoCent_images), len(cinquentaCent_images), len(umRealCent_images))


ModuleNotFoundError: No module named 'scipy'

In [ ]:
train_cincoCent_images = cincoCent_images[:int(.8*(len(cincoCent_images)))]
val_cincoCent_images = cincoCent_images[int(.8*(len(cincoCent_images))):]

train_dezCent_images = dezCent_images[:int(.8*(len(dezCent_images)))]
val_dezCent_images = dezCent_images[int(.8*(len(dezCent_images))):]

train_vinteCincoCent_images = vinteCincoCent_images[:int(.8*(len(vinteCincoCent_images)))]
val_vinteCincoCent_images = vinteCincoCent_images[int(.8*(len(vinteCincoCent_images))):]

train_cinquentaCent_images = cinquentaCent_images[:int(.8*(len(cinquentaCent_images)))]
val_cinquentaCent_images = cinquentaCent_images[int(.8*(len(cinquentaCent_images))):]

train_umRealCent_images = umRealCent_images[:int(.8*(len(umRealCent_images)))]
val_umRealCent_images = umRealCent_images[int(.8*(len(umRealCent_images))):]

# Gerando dados 
train_dir = 'train_data/'
val_dir = 'val_data/'

try:
    os.makedirs(os.path.join(base_dir, train_dir, '5/'))
except:
    pass
try:
    os.makedirs(os.path.join(base_dir, train_dir, '10/'))
except:
    pass
try:
    os.makedirs(os.path.join(base_dir, train_dir, '25/'))
except:
    pass
try:
    os.makedirs(os.path.join(base_dir, train_dir, '50/'))
except:
    pass
try:
    os.makedirs(os.path.join(base_dir, train_dir, '100/'))
except:
    pass

try:
    os.makedirs(os.path.join(base_dir, val_dir, '5/'))
except:
    pass
try:
    os.makedirs(os.path.join(base_dir, val_dir, '10/'))
except:
    pass
try:
    os.makedirs(os.path.join(base_dir, val_dir, '25/'))
except:
    pass
try:
    os.makedirs(os.path.join(base_dir, val_dir, '50/'))
except:
    pass
try:
    os.makedirs(os.path.join(base_dir, val_dir, '100/'))
except:
    pass

for image in train_cincoCent_images:
    src = CincoCentDir + image
    dst = os.path.join(base_dir, train_dir, '5/')
    shutil.copy(src, dst)

for image in train_dezCent_images:
    src = DezCentDir + image
    dst = os.path.join(base_dir, train_dir, '10/')
    shutil.copy(src, dst)

for image in train_vinteCincoCent_images:
    src = VinteCincoCentDir + image
    dst = os.path.join(base_dir, train_dir, '25/')
    shutil.copy(src, dst)

for image in train_cinquentaCent_images:
    src = CinquentaCentDir + image
    dst = os.path.join(base_dir, train_dir, '50/')
    shutil.copy(src, dst)

for image in train_umRealCent_images:
    src = UmRealCentDir + image
    dst = os.path.join(base_dir, train_dir, '100/')
    shutil.copy(src, dst)

for image in val_cincoCent_images:
    src = CincoCentDir + image
    dst = os.path.join(base_dir, val_dir, '5/')
    shutil.copy(src, dst)

for image in val_dezCent_images:
    src = DezCentDir + image
    dst = os.path.join(base_dir, val_dir, '10/')
    shutil.copy(src, dst)

for image in val_vinteCincoCent_images:
    src = VinteCincoCentDir + image
    dst = os.path.join(base_dir, val_dir, '25/')
    shutil.copy(src, dst)

for image in val_cinquentaCent_images:
    src = CinquentaCentDir + image
    dst = os.path.join(base_dir, val_dir, '50/')
    shutil.copy(src, dst)

for image in val_umRealCent_images:
    src = UmRealCentDir + image
    dst = os.path.join(base_dir, val_dir, '100/')
    shutil.copy(src, dst)



In [ ]:
train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.)
val_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.)

train_dataset = train_data_gen.flow_from_directory(os.path.join(base_dir, train_dir), target_size = (64,64), class_mode = 'categorical')
val_dataset = val_data_gen.flow_from_directory(os.path.join(base_dir, val_dir), target_size = (64,64), class_mode = 'categorical')


Found 2628 images belonging to 6 classes.
Found 660 images belonging to 6 classes.


In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='softmax'))
model.summary()


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 31, 31, 32)        0         
 g2D)                                                            
                                                                 
 flatten_8 (Flatten)         (None, 30752)             0         
                                                                 
 dense_16 (Dense)            (None, 64)                1968192   
                                                                 
 dense_17 (Dense)            (None, 5)                 325       
                                                                 
Total params: 1969413 (7.51 MB)
Trainable params: 1969413 (7.51 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.02)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['mse'])

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=100)

NameError: name 'scipy' is not defined

In [ ]:
""" 
plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.title("Função de perda")
plt.xlabel('Épocas')
plt.ylabel('MSE')
plt.legend(["Treinando"], loc='upper left')
plt.show() """
        

# Carrega o modelo treinado
# model.save(os.path.join(base_dir,'model_train.h5'))